In [96]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import layers

# Assignment 6: Cat and Dog recognition

In this assignment you will build a convolutional neural network (CNN) that is able to tell whether an image shows a cat or a dog. Furthermore, since we don’t have that many data to train our mode with, we will train another model that uses as a base the existing CNN model VGG16 that has frequently shown a very good performance in classifying images in datasets with hundreds or even thousands of possible classes. Lastly, you will evaluate the performance of the model and classify your own pet images. 

## Exercise 06.1: Training the CNN

What we want to do first is load all the images into arrays in RAM. The final goal should be a training set numpy array with a shape (23000,64, 64, 3) and a test set numpy array with a shape (2000, 64, 64, 3) containing all the images, where 23000 and 2000 are the total number of cat and dog images, 64x64 is the pixel size of each image and 3 is the number of color channels. You might not end up with a total of 25000 images because some images are corrupted. In any case, the test set should have 1000 cat images and 1000 dog images and the rest goes into the training set. Prepare these arrays, by resizing all of the images to 64x64 pixels and converting them into numpy arrays containing the RGB values of each pixel (convert the grey scale images accordingly). For this you might want to install and use the pillow library together with its Image functionalities such as resize and convert. Additionally, create two arrays with shapes (23000, 1) and (2000, 1) containing the corresponding labels (1 if the image shows a cat and 0 if it shows a dog).

Finally, normalize the RGB values that go from 0 to 255 to values between 0 and 1. Loading and preparing the arrays might take some time (if you want to monitor the progress, you can add progress bars with the tqdm library)!

Checking which images are corrupted:

In [72]:
for i in range(12500):
    try:
        Image.open(f"/Users/jercheal/Documents/Physics/CPIV_ML/PetImages/Cat/{i}.jpg")
    except:
        print(f"Cat image {i} is corrupted")

for i in range(12500):
    try:
        Image.open(f"/Users/jercheal/Documents/Physics/CPIV_ML/PetImages/Dog/{i}.jpg")
    except:
        print(f"Dog image {i} is corrupted")

Cat image 666 is corrupted
Dog image 11702 is corrupted


/opt/anaconda3/envs/CPIV/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Loading the images, resizing them, converting their RGB values into numpy arrays and normalizing by 250 such that the RGB values lie between 0 and 1.

In [79]:
cats_training = np.array([np.array(Image.open(f"/Users/jercheal/Documents/Physics/CPIV_ML/PetImages/Cat/{n}.jpg").resize((64, 64)).convert("RGB")) for n in range(11500) if n != 666]) /250
cats_test = np.array([np.array(Image.open(f"/Users/jercheal/Documents/Physics/CPIV_ML/PetImages/Cat/{n}.jpg").resize((64, 64)).convert("RGB")) for n in range(11500,12500)]) / 250
dogs_training = np.array([np.array(Image.open(f"/Users/jercheal/Documents/Physics/CPIV_ML/PetImages/Dog/{n}.jpg").resize((64, 64)).convert("RGB")) for n in range(11499)]) / 250
dogs_test = np.array([np.array(Image.open(f"/Users/jercheal/Documents/Physics/CPIV_ML/PetImages/Dog/{n}.jpg").resize((64, 64)).convert("RGB")) for n in range(11499,12500) if n != 11702]) / 250

/opt/anaconda3/envs/CPIV/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [121]:
training_data = np.concatenate((cats_training, dogs_training), axis=0)
training_labels = np.concatenate((np.ones(11499), np.zeros(11499)))
#test_data = np.concatenate((cats_test, dogs_test), axis=0)
test_labels = np.concatenate((np.ones(1000), np.zeros(1000)))
#training_data = np.expand_dims(training_data, -1)
test_data = np.expand_dims(test_data, -1)

In [123]:
training_data.shape

(22998, 64, 64, 3)

In [124]:
num_classes = 2
input_shape = (64, 64, 3)
training_labels = keras.utils.to_categorical(training_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)

In [125]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="sigmoid")
    ]
)

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │        25,090 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,482 (173.76 KB)

 Trainable params: 44,482 (173.76 KB)

 Non-trainable params: 0 (0.00 B)

In [127]:
batch_size = 64
epochs = 20

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(training_data, training_labels, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.7620 - loss: 0.4972 - val_accuracy: 0.6239 - val_loss: 0.6808
Epoch 2/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - accuracy: 0.7868 - loss: 0.4602 - val_accuracy: 0.7413 - val_loss: 0.5133
Epoch 3/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - accuracy: 0.7989 - loss: 0.4379 - val_accuracy: 0.7609 - val_loss: 0.4683
Epoch 4/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.8062 - loss: 0.4345 - val_accuracy: 0.6343 - val_loss: 0.6752
Epoch 5/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.8193 - loss: 0.4065 - val_accuracy: 0.7070 - val_loss: 0.5542
Epoch 6/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.8216 - loss: 0.3942 - val_accuracy: 0.8439 - val_loss: 0.3485
Epoch 7/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.8319 - loss: 0.3831 - val_accuracy: 0.8009 - val_loss: 0.4261
Epoch 8/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.8345 - loss: 0.3757 - va

In [129]:
test_labels.shape

(2000, 2)

In [128]:
score = model.evaluate(test_data, test_labels, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None